In [270]:
#Added Libraries 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys #For accessing buttons
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import time
import pandas as pd
import math

In [271]:
#city = input("Enter a city:") #Get input from user
city = 'New York'
count = int(input("Enter number of restraurants required:"))

Enter number of restraurants required:5


In [272]:
#Calculated the number of pages to be scraped (Google map displayes 20 results per page)
pages = math.ceil(count/20)
print(pages)

1


In [273]:
df= pd.DataFrame()

In [274]:
browser = webdriver.Chrome('chromedriver')  # opens browser

In [275]:
#Initialized webdriver

browser.get("https://www.google.com/maps") #pass URL

search_form = browser.find_element_by_xpath('//*[@id="searchboxinput"]')
time.sleep(2) #waits
search_text = city + ' ' +"restaurants"
search_form.send_keys(search_text)

time.sleep(2) #waits
search_form.send_keys(Keys.ENTER)


In [276]:
#Automating Google Map search and extraction

x=0
R_name=[]
R_address=[]
for j in range(pages):
    for i in range(1,count*2,2):
        x=x+1
        name_xpath = '//*[@id="pane"]/div/div[1]/div/div/div[4]/div[1]/div['+str(i)+']/div[1]/div[1]/div[1]/div[1]/div[2]/h3'
        page_xpath = '//*[@id="pane"]/div/div[1]/div/div/div[4]/div[1]/div['+str(i)+']/div[1]/div[1]'
        try:
            name = browser.find_element_by_xpath(name_xpath).text
            browser.find_element_by_xpath(page_xpath).click()
            time.sleep(3)
            address = browser.find_element_by_class_name("ugiz4pqJLAG__text").text 
            R_name.append(name)
            R_address.append(address)
            browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/button').click()
            time.sleep(3)
        except NoSuchElementException:
            pass
        
        if (x%20 == 0): break
    print("Page ",j+1," ends")
    browser.find_element_by_xpath('//*[@id="n7lv7yjyC35__section-pagination-button-next"]').click() #Click next
    time.sleep(3) 


Page  1  ends


In [277]:
R_name

['MARKET CRATES',
 'Midtown Social',
 'Le Bernardin',
 'The Modern',
 'LOS TACOS No. 1']

In [278]:
R_address

['224 W 35th St, New York, NY 10001, United States',
 '15 W 39th St, New York, NY 10018, United States',
 '155 W 51st St, New York, NY 10019, United States',
 '9 W 53rd St, New York, NY 10019, United States',
 '75 9th Ave, New York, NY 10011, United States']

In [285]:
# Restaurant URL extraction 

from fuzzywuzzy import fuzz
from selenium.webdriver.common.keys import Keys
#browser.get("https://www.google.com/")
url_list=[]
 
for name in R_name:
    browser.get("https://www.google.com/")
    search_form = browser.find_element_by_xpath('//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input')
    time.sleep(2)
    search_text = name + ' ' + city + ' website'
    try:
        search_form.send_keys(search_text) #Keywords to search website
        time.sleep(2) #waits
        print("Done")
        search_form.send_keys(Keys.ENTER)
    except StaleElementReferenceException: 
        pass
    website_info=[]
    for i in range(10):
        try:
            name_xpath= '//*[@id="rso"]/div['+str(i)+']/div/div[1]/a/h3/span'
            name_url = browser.find_element_by_xpath(name_xpath).text
            score = fuzz.token_set_ratio(name, name_url)  # (levenshtein distance)
            website_info.append({'xpath_id' : i, 'score' : score})
        except NoSuchElementException:
            pass
        
    sorted_score_dict = sorted(website_info, key = lambda i: i["score"], reverse = True)
    print(sorted_score_dict)
    if sorted_score_dict[0]['score']:
        xpath_id = sorted_score_dict[0]['xpath_id']    
        try:
            url_xpath = '//*[@id="rso"]/div['+str(xpath_id)+']/div/div[1]/a/div/cite'
            url = browser.find_element_by_xpath(url_xpath).text
        except NoSuchElementException:
            pass
    else:
        url = 'Unavailable'
        print("No")
    url_list.append(url)
    

    

Done
[{'xpath_id': 1, 'score': 100}, {'xpath_id': 2, 'score': 100}, {'xpath_id': 4, 'score': 100}, {'xpath_id': 5, 'score': 100}, {'xpath_id': 6, 'score': 100}, {'xpath_id': 7, 'score': 100}, {'xpath_id': 8, 'score': 100}, {'xpath_id': 9, 'score': 100}]
Done
[{'xpath_id': 1, 'score': 100}, {'xpath_id': 2, 'score': 100}, {'xpath_id': 4, 'score': 100}, {'xpath_id': 5, 'score': 100}, {'xpath_id': 6, 'score': 100}, {'xpath_id': 7, 'score': 100}, {'xpath_id': 9, 'score': 100}, {'xpath_id': 3, 'score': 74}]
Done
[{'xpath_id': 1, 'score': 100}, {'xpath_id': 2, 'score': 100}, {'xpath_id': 4, 'score': 100}, {'xpath_id': 5, 'score': 100}, {'xpath_id': 7, 'score': 100}, {'xpath_id': 8, 'score': 100}, {'xpath_id': 9, 'score': 100}]
Done
[{'xpath_id': 1, 'score': 100}, {'xpath_id': 2, 'score': 100}, {'xpath_id': 3, 'score': 100}, {'xpath_id': 6, 'score': 100}, {'xpath_id': 7, 'score': 100}, {'xpath_id': 8, 'score': 100}, {'xpath_id': 9, 'score': 100}, {'xpath_id': 5, 'score': 29}]
Done
[{'xpath_id'

In [286]:
url_list

['www.marketcrates.com',
 'www.facebook.com › Pages › Public Figure › Artist',
 'www.le-bernardin.com',
 'www.themodernnyc.com',
 'www.lostacos1.com']

In [281]:
#Storing data

df['Restaurant_name'] = R_name
df['Address'] = R_address
df['URL'] = url_list

In [282]:
df

,Restaurant_name,Address,URL
0,MARKET CRATES,"224 W 35th St, New York, NY 10001, United States",www.marketcrates.com
1,Midtown Social,"15 W 39th St, New York, NY 10018, United States",www.yelp.com › Restaurants › Mexican › Tacos
2,Le Bernardin,"155 W 51st St, New York, NY 10019, United States",www.le-bernardin.com
3,The Modern,"9 W 53rd St, New York, NY 10019, United States",www.themodernnyc.com
4,LOS TACOS No. 1,"75 9th Ave, New York, NY 10011, United States",www.lostacos1.com
